In [143]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [144]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [145]:
#2.1 Get the file
downloaded = drive.CreateFile({'id':'1GOpbxiRSQbg9ESCt5ufiqkBRTa1H88h4'}) # replace the id with id of file you want to access
downloaded.GetContentFile('1807242.csv') 

Importing all the libraries 


In [146]:


import numpy as np
import pandas as pd
import datetime
import statistics
import seaborn as sns
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [147]:
#Importing the dataset
data=pd.read_csv('1807242.csv')
data.head(4)

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,200769623,WAL-MAR systems,04-06-2019 00:00,2019,1929359463,22-05-2019,20190522,20190522,20190606,USD,RV,1,NaN,14916.57,20190522,NAH4,1.929359e+09,0
1,U001,200769623,WAL-MAR corporation,13-08-2019 00:00,2019,1929690827,02-08-2019,20190802,20190802,20190817,USD,RV,1,NaN,17878.43,20190802,NAH4,1.929691e+09,0
2,U001,200353024,DECA foundation,12-04-2019 00:00,2019,1929135118,12-04-2019,20190412,20190412,20190408,USD,RV,1,NaN,5709.12,20190316,NAM4,1.929135e+09,0
3,U001,200744019,TARG trust,09-05-2019 00:00,2019,1929194057,24-04-2019,20190423,20190424,20190509,USD,RV,1,NaN,3590.92,20190424,NAA8,1.929194e+09,0


In [148]:
#dropping column with all null
data.dropna(axis=1,how="all", inplace=True)

data.head(5)

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date1,due_in_date,invoice_currency,document type,posting_id,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,200769623,WAL-MAR systems,04-06-2019 00:00,2019,1929359463,22-05-2019,20190522,20190522,20190606,USD,RV,1,14916.57,20190522,NAH4,1.929359e+09,0
1,U001,200769623,WAL-MAR corporation,13-08-2019 00:00,2019,1929690827,02-08-2019,20190802,20190802,20190817,USD,RV,1,17878.43,20190802,NAH4,1.929691e+09,0
2,U001,200353024,DECA foundation,12-04-2019 00:00,2019,1929135118,12-04-2019,20190412,20190412,20190408,USD,RV,1,5709.12,20190316,NAM4,1.929135e+09,0
3,U001,200744019,TARG trust,09-05-2019 00:00,2019,1929194057,24-04-2019,20190423,20190424,20190509,USD,RV,1,3590.92,20190424,NAA8,1.929194e+09,0
4,U001,200739941,FOX,15-01-2020 00:00,2020,1930329674,30-12-2019,20191228,20191230,20200114,USD,RV,1,21390.29,20191230,NAA8,1.930330e+09,0


In [149]:
#to find the amount of null data in each column
data.isna().sum()

business_code               0
cust_number                 0
name_customer               0
clear_date               3762
buisness_year               0
doc_id                      0
posting_date                0
document_create_date        0
document_create_date1       0
due_in_date                 0
invoice_currency            0
document type               0
posting_id                  0
total_open_amount           0
baseline_create_date        0
cust_payment_terms          0
invoice_id                  5
isOpen                      0
dtype: int64

In [150]:
#dropping the corelated columns
corr_matrix= data.corr().abs()
upper= corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop= [column for column in upper.columns if any(upper[column]>0.95)]
data.drop(data[to_drop], axis=1)

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,invoice_currency,document type,posting_id,total_open_amount,cust_payment_terms,isOpen
0,U001,200769623,WAL-MAR systems,04-06-2019 00:00,2019,1929359463,22-05-2019,USD,RV,1,14916.57,NAH4,0
1,U001,200769623,WAL-MAR corporation,13-08-2019 00:00,2019,1929690827,02-08-2019,USD,RV,1,17878.43,NAH4,0
2,U001,200353024,DECA foundation,12-04-2019 00:00,2019,1929135118,12-04-2019,USD,RV,1,5709.12,NAM4,0
3,U001,200744019,TARG trust,09-05-2019 00:00,2019,1929194057,24-04-2019,USD,RV,1,3590.92,NAA8,0
4,U001,200739941,FOX,15-01-2020 00:00,2020,1930329674,30-12-2019,USD,RV,1,21390.29,NAA8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,U001,200769623,WAL-MAR systems,NaN,2020,1930796703,17-04-2020,USD,RV,1,8645.54,NAH4,1
49996,CA02,140106408,WAL-M corporation,NaN,2020,2960632401,30-04-2020,CAD,RV,1,12996.08,CA10,1
49997,U001,200803720,DEC llc,NaN,2020,1930691770,24-03-2020,USD,RV,1,6037.62,NAM2,1
49998,U001,100031704,DELHAIZE trust,NaN,2020,1930646387,16-03-2020,USD,RV,1,87552.87,NAA8,1


In [151]:
data.nunique()

business_code                6
cust_number               1110
name_customer             4256
clear_date                 404
buisness_year                2
doc_id                   50000
posting_date               502
document_create_date       505
document_create_date1      504
due_in_date                541
invoice_currency             2
document type                2
posting_id                   1
total_open_amount        45141
baseline_create_date       506
cust_payment_terms          75
invoice_id               49995
isOpen                       2
dtype: int64

In [152]:
#dropping quasi constant column
data=data.drop(['posting_id'], axis = 1)
data=data.drop(['invoice_currency'], axis = 1)
data.shape

(50000, 16)

In [153]:
#for each business_code eixts a unique cus_number
data=data.drop(['business_code'], axis = 1)
data.shape

(50000, 15)

In [154]:
data.dtypes

cust_number               object
name_customer             object
clear_date                object
buisness_year              int64
doc_id                     int64
posting_date              object
document_create_date       int64
document_create_date1      int64
due_in_date                int64
document type             object
total_open_amount        float64
baseline_create_date       int64
cust_payment_terms        object
invoice_id               float64
isOpen                     int64
dtype: object

Conversion to datetime

In [155]:
data['due_in_date']= pd.to_datetime(data.due_in_date, format='%Y%m%d')
data.head()

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date1,due_in_date,document type,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,200769623,WAL-MAR systems,04-06-2019 00:00,2019,1929359463,22-05-2019,20190522,20190522,2019-06-06,RV,14916.57,20190522,NAH4,1.929359e+09,0
1,200769623,WAL-MAR corporation,13-08-2019 00:00,2019,1929690827,02-08-2019,20190802,20190802,2019-08-17,RV,17878.43,20190802,NAH4,1.929691e+09,0
2,200353024,DECA foundation,12-04-2019 00:00,2019,1929135118,12-04-2019,20190412,20190412,2019-04-08,RV,5709.12,20190316,NAM4,1.929135e+09,0
3,200744019,TARG trust,09-05-2019 00:00,2019,1929194057,24-04-2019,20190423,20190424,2019-05-09,RV,3590.92,20190424,NAA8,1.929194e+09,0
4,200739941,FOX,15-01-2020 00:00,2020,1930329674,30-12-2019,20191228,20191230,2020-01-14,RV,21390.29,20191230,NAA8,1.930330e+09,0


In [156]:
data['clear_date']= pd.to_datetime(data["clear_date"])


In [157]:
data['document_create_date1']= pd.to_datetime(data.document_create_date1, format='%Y%m%d')
data.head()

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date1,due_in_date,document type,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,200769623,WAL-MAR systems,2019-04-06,2019,1929359463,22-05-2019,20190522,2019-05-22,2019-06-06,RV,14916.57,20190522,NAH4,1.929359e+09,0
1,200769623,WAL-MAR corporation,2019-08-13,2019,1929690827,02-08-2019,20190802,2019-08-02,2019-08-17,RV,17878.43,20190802,NAH4,1.929691e+09,0
2,200353024,DECA foundation,2019-12-04,2019,1929135118,12-04-2019,20190412,2019-04-12,2019-04-08,RV,5709.12,20190316,NAM4,1.929135e+09,0
3,200744019,TARG trust,2019-09-05,2019,1929194057,24-04-2019,20190423,2019-04-24,2019-05-09,RV,3590.92,20190424,NAA8,1.929194e+09,0
4,200739941,FOX,2020-01-15,2020,1930329674,30-12-2019,20191228,2019-12-30,2020-01-14,RV,21390.29,20191230,NAA8,1.930330e+09,0


In [158]:
#since posting date can provide better info
data['posting_date']= pd.to_datetime(data['posting_date'])
data.drop('document_create_date', axis=1, inplace=True)
#data.drop('document_create_date1', axis=1, inplace=True)
data.head()

,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date1,due_in_date,document type,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,200769623,WAL-MAR systems,2019-04-06,2019,1929359463,2019-05-22,2019-05-22,2019-06-06,RV,14916.57,20190522,NAH4,1.929359e+09,0
1,200769623,WAL-MAR corporation,2019-08-13,2019,1929690827,2019-02-08,2019-08-02,2019-08-17,RV,17878.43,20190802,NAH4,1.929691e+09,0
2,200353024,DECA foundation,2019-12-04,2019,1929135118,2019-12-04,2019-04-12,2019-04-08,RV,5709.12,20190316,NAM4,1.929135e+09,0
3,200744019,TARG trust,2019-09-05,2019,1929194057,2019-04-24,2019-04-24,2019-05-09,RV,3590.92,20190424,NAA8,1.929194e+09,0
4,200739941,FOX,2020-01-15,2020,1930329674,2019-12-30,2019-12-30,2020-01-14,RV,21390.29,20191230,NAA8,1.930330e+09,0


In [159]:

data.dtypes

cust_number                      object
name_customer                    object
clear_date               datetime64[ns]
buisness_year                     int64
doc_id                            int64
posting_date             datetime64[ns]
document_create_date1    datetime64[ns]
due_in_date              datetime64[ns]
document type                    object
total_open_amount               float64
baseline_create_date              int64
cust_payment_terms               object
invoice_id                      float64
isOpen                            int64
dtype: object

In [160]:
data=data.astype({"cust_number" : 'category'})
data.drop('name_customer', axis=1, inplace=True)
data.dtypes


cust_number                    category
clear_date               datetime64[ns]
buisness_year                     int64
doc_id                            int64
posting_date             datetime64[ns]
document_create_date1    datetime64[ns]
due_in_date              datetime64[ns]
document type                    object
total_open_amount               float64
baseline_create_date              int64
cust_payment_terms               object
invoice_id                      float64
isOpen                            int64
dtype: object

In [161]:
#converting clear_date to datetime format
data['clear_date']=pd.to_datetime(data['clear_date'])

In [162]:
#calculating delay
data['delay']=(data['clear_date']-data['due_in_date']).dt.days
data.head(20)

,cust_number,clear_date,buisness_year,doc_id,posting_date,document_create_date1,due_in_date,document type,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,delay
0,200769623,2019-04-06,2019,1929359463,2019-05-22,2019-05-22,2019-06-06,RV,14916.57,20190522,NAH4,1.929359e+09,0,-61.0
1,200769623,2019-08-13,2019,1929690827,2019-02-08,2019-08-02,2019-08-17,RV,17878.43,20190802,NAH4,1.929691e+09,0,-4.0
2,200353024,2019-12-04,2019,1929135118,2019-12-04,2019-04-12,2019-04-08,RV,5709.12,20190316,NAM4,1.929135e+09,0,240.0
3,200744019,2019-09-05,2019,1929194057,2019-04-24,2019-04-24,2019-05-09,RV,3590.92,20190424,NAA8,1.929194e+09,0,119.0
4,200739941,2020-01-15,2020,1930329674,2019-12-30,2019-12-30,2020-01-14,RV,21390.29,20191230,NAA8,1.930330e+09,0,1.0
5,200780383,2019-07-06,2019,1929076457,2019-03-04,2019-04-03,2019-06-07,RV,1299.19,20190403,NAGD,1.929076e+09,0,29.0
6,200729290,2019-12-12,2019,1930094244,2019-02-11,2019-11-02,2019-12-12,RV,63021.51,20191102,NAWU,1.930094e+09,0,0.0
7,200794332,2019-07-08,2019,1929651773,2019-07-24,2019-07-24,2019-08-08,RV,1508.48,20190724,NAAX,1.929652e+09,0,-31.0
8,200769623,2019-09-12,2019,1930199514,2019-11-27,2019-11-27,2019-12-12,RV,7358.49,20191127,NAH4,1.930200e+09,0,-91.0
9,200792734,2019-10-29,2019,1930009122,2019-10-14,2019-10-14,2019-10-29,RV,155210.96,20191014,NAA8,1.930009e+09,0,0.0


In [163]:
#converting the delay to int type
type(data['delay'])
data.dtypes

cust_number                    category
clear_date               datetime64[ns]
buisness_year                     int64
doc_id                            int64
posting_date             datetime64[ns]
document_create_date1    datetime64[ns]
due_in_date              datetime64[ns]
document type                    object
total_open_amount               float64
baseline_create_date              int64
cust_payment_terms               object
invoice_id                      float64
isOpen                            int64
delay                           float64
dtype: object

In [164]:
#dropping cust_number
data.drop('cust_number', axis=1, inplace=True)


In [165]:
#dropping cust_payment_terms
data.drop('cust_payment_terms', axis=1, inplace=True)

In [166]:
#dropping document_type
data.drop('document type', axis=1, inplace=True)

In [167]:
#dropping total_open_amount
data.drop('total_open_amount', axis=1, inplace=True)

In [168]:
#dropping posting date
data.drop('posting_date', axis=1, inplace=True)

Splitting the Dataset into Train set and test set on the basis of clear_date

In [169]:
X_train= data[data['clear_date'].notnull()]

X_test= data[data['clear_date'].isnull()]

In [170]:
X_train = X_train.sort_values(by='document_create_date1')

In [171]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)

print('correlated features: ', corr_features)

correlated features:  {'baseline_create_date', 'invoice_id'}


In [172]:
X_train.head(30)

,clear_date,buisness_year,doc_id,document_create_date1,due_in_date,baseline_create_date,invoice_id,isOpen,delay
390,2019-12-27,2019,9500000065,2018-11-20,2018-11-30,20181120,NaN,0,392.0
8256,2019-12-27,2019,9500000067,2018-12-04,2018-12-14,20181204,NaN,0,378.0
15719,2019-01-15,2019,1928541234,2018-12-30,2019-01-14,20181230,1.928541e+09,0,1.0
14672,2019-10-01,2019,1928539666,2018-12-30,2019-01-14,20181230,1.928540e+09,0,260.0
6129,2019-09-01,2019,1928516217,2018-12-30,2019-01-14,20181230,1.928516e+09,0,230.0
19370,2019-09-01,2019,1928538576,2018-12-30,2019-01-14,20181230,1.928539e+09,0,230.0
34983,2019-11-01,2019,1928539722,2018-12-30,2019-01-14,20181230,1.928540e+09,0,291.0
34944,2019-01-15,2019,1928539311,2018-12-30,2019-01-14,20181230,1.928539e+09,0,1.0
21548,2019-09-01,2019,1928541355,2018-12-30,2019-01-14,20181230,1.928541e+09,0,230.0
29520,2019-09-01,2019,1928537307,2018-12-30,2019-01-14,20181230,1.928537e+09,0,230.0


In [173]:
X_train.isna().sum()

clear_date               0
buisness_year            0
doc_id                   0
document_create_date1    0
due_in_date              0
baseline_create_date     0
invoice_id               5
isOpen                   0
delay                    0
dtype: int64

In [174]:
#Conversion of datetime objects as it can't be fed to model
X_train['clear_date']=pd.to_numeric(X_train['clear_date'])


In [175]:
X_train['due_in_date']=pd.to_numeric(X_train['due_in_date'])


In [176]:
X_train['document_create_date1']=pd.to_numeric(X_train['document_create_date1'])
X_train.dtypes

clear_date                 int64
buisness_year              int64
doc_id                     int64
document_create_date1      int64
due_in_date                int64
baseline_create_date       int64
invoice_id               float64
isOpen                     int64
delay                    float64
dtype: object

In [177]:
#dropping invoive_id
X_train.drop('invoice_id', axis=1, inplace=True)

In [178]:
X_test.drop('invoice_id', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [179]:
#traget variable
y_train=X_train['delay']


In [180]:
X_train.drop('delay', axis=1, inplace=True)

In [181]:
y_train.shape

(46238,)

In [182]:
X_test.dtypes

clear_date               datetime64[ns]
buisness_year                     int64
doc_id                            int64
document_create_date1    datetime64[ns]
due_in_date              datetime64[ns]
baseline_create_date              int64
isOpen                            int64
delay                           float64
dtype: object

In [183]:
X_train.shape

(46238, 7)

In [184]:
#changing datetime objects in test set
X_test['clear_date']=pd.to_numeric(X_test['clear_date'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [185]:
#converting to numeric because datetime object cannot be fed to model
X_test['document_create_date1']=pd.to_numeric(X_test['document_create_date1'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [186]:
#converting to numeric because datetime object cannot be fed to model
X_test['due_in_date']=pd.to_numeric(X_test['due_in_date'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [187]:
X_test.dtypes

clear_date                 int64
buisness_year              int64
doc_id                     int64
document_create_date1      int64
due_in_date                int64
baseline_create_date       int64
isOpen                     int64
delay                    float64
dtype: object

In [188]:
X_test.isna().sum()

clear_date                  0
buisness_year               0
doc_id                      0
document_create_date1       0
due_in_date                 0
baseline_create_date        0
isOpen                      0
delay                    3762
dtype: int64

Splitting the train set into 2 validation set

In [189]:
x_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size= .20, shuffle=False, random_state=1)
X_valid1, X_valid2, y_valid1, y_valid2 = train_test_split(X_valid, y_valid, test_size=.50, shuffle=False, random_state=1)

In [190]:
# Defining Lists to Store in the Results and Names of Algorithms
MSE_Score = []
R2_Score = []
Algorithm = []

Using random forest regressor

In [191]:
# Fitting Random Forest Regressor Tree to the Training Set
Algorithm.append('Random Forest Regressor')
clf = RandomForestRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(x_train)

In [192]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_train, predicted))
R2_Score.append(r2_score(y_train, predicted))

In [193]:
# Fitting Random Forest Regressor Tree to the Training Set
Algorithm.append('Random Forest Regressor')
clf = RandomForestRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid1)

In [194]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid1, predicted))
R2_Score.append(r2_score(y_valid1, predicted))

In [195]:
# Fitting Random Forest Regressor Tree to the Training Set
Algorithm.append('Random Forest Regressor')
clf = RandomForestRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid2)

In [196]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid2, predicted))
R2_Score.append(r2_score(y_valid2, predicted))


Linear regression

In [197]:
# Fitting Simple Linear Regression to the Training Set
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(x_train)

In [198]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_train, predicted))
R2_Score.append(r2_score(y_train, predicted))

In [199]:
# Fitting Simple Linear Regression to the validation set1
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid1)

In [200]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid1, predicted))
R2_Score.append(r2_score(y_valid1, predicted))


In [201]:
# Fitting Simple Linear Regression to the validation set1
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid2)

In [202]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid2, predicted))
R2_Score.append(r2_score(y_valid2, predicted))


XGB regressor

In [203]:
# Fitting XGBoost Regressor to the Training Set
Algorithm.append('XGB Regressor')
clf = xgb.XGBRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(x_train)

[06:58:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [204]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_train, predicted))
R2_Score.append(r2_score(y_train, predicted))

In [205]:
# Fitting XGBoost Regressor to the validation Set1
Algorithm.append('XGB Regressor')
clf = xgb.XGBRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid1)

[06:58:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [206]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid1, predicted))
R2_Score.append(r2_score(y_valid1, predicted))


In [207]:
# Fitting XGBoost Regressor to the validation set2
Algorithm.append('XGB Regressor')
clf = xgb.XGBRegressor()
clf.fit(x_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_valid2)

[06:58:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [208]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_valid2, predicted))
R2_Score.append(r2_score(y_valid2, predicted))

In [209]:
# Just Combining the Lists into a DataFrame for a Better Visualisation
Comparison = pd.DataFrame(list(zip(Algorithm, MSE_Score, R2_Score)), columns = ['Algorithm', 'MSE_Score', 'R2_Score'])

In [210]:
# Score Card
Comparison

,Algorithm,MSE_Score,R2_Score
0,Random Forest Regressor,1.766410e-01,0.999978
1,Random Forest Regressor,1.960155e+03,0.715431
2,Random Forest Regressor,1.558572e+04,-0.800691
3,Linear Regression,6.780987e-23,1.000000
4,Linear Regression,6.780960e-23,1.000000
5,Linear Regression,6.782144e-23,1.000000
6,XGB Regressor,2.449387e+01,0.996908
7,XGB Regressor,1.646220e+03,0.761007
8,XGB Regressor,1.256219e+04,-0.451369


Creating bucket

In [211]:
#making a list named as age_bucket
age_bucket=[]
for row in data['delay']:
        if row <= 15: age_bucket.append('0-15')
        elif row<=30: age_bucket.append('16-30')
        elif row <= 45: age_bucket.append('31-45')
        elif row <=60:  age_bucket.append('46-60')
         
        else:  age_bucket.append('More than 60 days')

data['age_bucket']= age_bucket        

In [212]:
data.head(6)

,clear_date,buisness_year,doc_id,document_create_date1,due_in_date,baseline_create_date,invoice_id,isOpen,delay,age_bucket
0,2019-04-06,2019,1929359463,2019-05-22,2019-06-06,20190522,1.929359e+09,0,-61.0,0-15
1,2019-08-13,2019,1929690827,2019-08-02,2019-08-17,20190802,1.929691e+09,0,-4.0,0-15
2,2019-12-04,2019,1929135118,2019-04-12,2019-04-08,20190316,1.929135e+09,0,240.0,More than 60 days
3,2019-09-05,2019,1929194057,2019-04-24,2019-05-09,20190424,1.929194e+09,0,119.0,More than 60 days
4,2020-01-15,2020,1930329674,2019-12-30,2020-01-14,20191230,1.930330e+09,0,1.0,0-15
5,2019-07-06,2019,1929076457,2019-04-03,2019-06-07,20190403,1.929076e+09,0,29.0,16-30
